In [ ]:
import pandas as pd
import FinanceDataReader as fdr
from datetime import datetime
from tqdm import tqdm
import warnings
import time
import os
import numpy as np
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [ ]:
# 경로
list_path = '/home/dave/projects/dev_backend/데이터/README/list.csv'

print(f"Loading tickers from: {list_path}")

# 한글 지원을 위해 encoding cp949 사용
ticker_df = pd.read_csv(list_path, encoding='cp949')

print(f"✓ Successfully loaded {len(ticker_df)} stocks")
print("\nFirst few rows:")
print(ticker_df.head())

# 종목 ticker코드를 0을 사용해서 6자 유지
tickers = ticker_df.iloc[:, 0].astype(str).str.zfill(6).tolist()

print(f"\n✓ Extracted {len(tickers)} tickers")
print(f"First 5 tickers: {tickers[:5]}")

Loading tickers from: /home/dave/projects/dev_backend/데이터/README/list.csv
✓ Successfully loaded 200 stocks

First few rows:
    종목코드     종목명       종가    대비   등락률       상장시가총액
0   5930    삼성전자  1330000  3000  0.23  195908118.0
1   5380     현대차   169000     0  0.00   37226725.0
2    660  SK하이닉스    47750     0  0.00   34762113.0
3  15760    한국전력    42700     0  0.00   27411866.0
4   5490   POSCO   283500  8000  2.90   24717468.0

✓ Extracted 200 tickers
First 5 tickers: ['005930', '005380', '000660', '015760', '005490']


In [ ]:
# 날짜 설정
start_date = '2015-01-01'
end_date = '2024-12-31'

print(f"Data collection period: {start_date} to {end_date}")
print(f"Number of stocks to download: {len(tickers)}")

Data collection period: 2015-01-01 to 2024-12-31
Number of stocks to download: 200


In [ ]:
# 종목 OHLCV
all_data = []
failed_tickers = []

print("Starting data collection...\n")

for ticker in tqdm(tickers):
    try:
        df = fdr.DataReader(ticker, start_date, end_date)
        
        if not df.empty:
            # Ticker column 추가
            df['Ticker'] = ticker
            df = df.reset_index()
            
            all_data.append(df)
        else:
            failed_tickers.append(ticker)
            
    except Exception as e:
        failed_tickers.append(ticker)
        print(f"Error: {ticker} - {e}")

print(f"\n✓ Downloaded: {len(all_data)} stocks")
print(f"✗ Failed: {len(failed_tickers)} stocks")

Starting data collection...



100%|██████████| 200/200 [00:15<00:00, 12.61it/s]


✓ Downloaded: 200 stocks
✗ Failed: 0 stocks


In [ ]:
# DataFrame 하나로 정리
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Ticker & Date로 정리
    combined_df = combined_df.sort_values(['Ticker', 'Date']).reset_index(drop=True)
    
    print(f"Combined DataFrame shape: {combined_df.shape}")
    print(f"\nDate range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"Number of unique stocks: {combined_df['Ticker'].nunique()}")
    print(f"\nFirst few rows:")
    print(combined_df.head(10))
else:
    print("No data collected!")

Combined DataFrame shape: (471300, 8)

Date range: 2015-01-02 00:00:00 to 2024-12-30 00:00:00
Number of unique stocks: 200

First few rows:
        Date   Open   High   Low  Close   Volume    Change  Ticker
0 2015-01-02  10000  10000  9760   9790  2918197 -0.021000  000030
1 2015-01-05   9750   9760  9530   9630  3563551 -0.016343  000030
2 2015-01-06   9520   9580  9410   9440  2619991 -0.019730  000030
3 2015-01-07   9380   9510  9370   9400  1679442 -0.004237  000030
4 2015-01-08   9420   9620  9410   9530  2166767  0.013830  000030
5 2015-01-09   9630   9660  9510   9560  1737936  0.003148  000030
6 2015-01-12   9500   9540  9360   9380  1449448 -0.018828  000030
7 2015-01-13   9400   9490  9320   9390  1380551  0.001066  000030
8 2015-01-14   9390   9420  9190   9220  2399519 -0.018104  000030
9 2015-01-15   9190   9200  9000   9050  2884385 -0.018438  000030


In [ ]:
print("Calculating rolling returns...\n")

# Rolling window 설정
return_windows = [1, 5, 20, 30, 50, 60, 100, 120, 200]

all_stocks_with_returns = []

for ticker in tqdm(combined_df['Ticker'].unique(), desc="Calculating returns"):
    # 종목별 계산
    stock_df = combined_df[combined_df['Ticker'] == ticker].copy()
    stock_df = stock_df.sort_values('Date').reset_index(drop=True)
    
    # Rolling window 설정 계산
    for window in return_windows:
        # Return = (오늘 종가 - N일 전 종가) / N일 전 종가 * 100
        stock_df[f'Return_{window}d'] = stock_df['Close'].pct_change(periods=window) * 100
        
        # 첫 N일, fill NaN 0으로
        stock_df[f'Return_{window}d'] = stock_df[f'Return_{window}d'].fillna(0)
    
    all_stocks_with_returns.append(stock_df)

# 모든 종목 수익률 계산
combined_df_with_returns = pd.concat(all_stocks_with_returns, ignore_index=True)

print(f"\n✓ Added rolling return features")
print(f"Return columns: {[col for col in combined_df_with_returns.columns if 'Return_' in col]}")
print(f"\nDataFrame shape: {combined_df_with_returns.shape}")
print(f"Total columns: {len(combined_df_with_returns.columns)}")

# 샘플 데이터
print(f"\n{'='*80}")
print(f"Sample: First 10 rows of first stock")
print(f"{'='*80}")
sample = combined_df_with_returns[combined_df_with_returns['Ticker'] == combined_df_with_returns['Ticker'].iloc[0]].head(10)
print(sample[['Date', 'Close', 'Return_1d', 'Return_5d', 'Return_20d', 'Return_60d']])

Calculating rolling returns...



Calculating returns: 100%|██████████| 200/200 [00:04<00:00, 42.21it/s]



✓ Added rolling return features
Return columns: ['Return_1d', 'Return_5d', 'Return_20d', 'Return_30d', 'Return_50d', 'Return_60d', 'Return_100d', 'Return_120d', 'Return_200d']

DataFrame shape: (471300, 17)
Total columns: 17

Sample: First 10 rows of first stock
        Date  Close  Return_1d  Return_5d  Return_20d  Return_60d
0 2015-01-02   9790   0.000000   0.000000         0.0         0.0
1 2015-01-05   9630  -1.634321   0.000000         0.0         0.0
2 2015-01-06   9440  -1.973001   0.000000         0.0         0.0
3 2015-01-07   9400  -0.423729   0.000000         0.0         0.0
4 2015-01-08   9530   1.382979   0.000000         0.0         0.0
5 2015-01-09   9560   0.314795  -2.349336         0.0         0.0
6 2015-01-12   9380  -1.882845  -2.596054         0.0         0.0
7 2015-01-13   9390   0.106610  -0.529661         0.0         0.0
8 2015-01-14   9220  -1.810437  -1.914894         0.0         0.0
9 2015-01-15   9050  -1.843818  -5.036726         0.0         0.0


In [ ]:
print("Calculating volume pattern features...\n")

all_stocks_with_volume = []

for ticker in tqdm(combined_df_with_returns['Ticker'].unique(), desc="Adding volume features"):
    # 종목별 계산
    stock_df = combined_df_with_returns[combined_df_with_returns['Ticker'] == ticker].copy()
    stock_df = stock_df.sort_values('Date').reset_index(drop=True)
    
    # 1. vol_20 - 20-day rolling volatility
    stock_df['vol_20'] = stock_df['Close'].pct_change().rolling(window=20, min_periods=1).std()
    
    # 2. vol_60 - 60-day rolling volatility
    stock_df['vol_60'] = stock_df['Close'].pct_change().rolling(window=60, min_periods=1).std()
    
    # 3. vol_60_sqrt252 - 60-day volatility * sqrt(252) for annualization
    stock_df['vol_60_sqrt252'] = stock_df['vol_60'] * np.sqrt(252)
    
    # 4. log_vol - Log of volume
    stock_df['log_vol'] = np.log(stock_df['Volume'] + 1)  # +1 to avoid log(0)
    
    # 5. vol_ratio_60 - Log volume ratio vs 60-day mean
    log_vol_mean_60 = stock_df['log_vol'].rolling(window=60, min_periods=1).mean()
    stock_df['vol_ratio_60'] = stock_df['log_vol'] - log_vol_mean_60
    
    # 6. avg_log_vol_ratio_60 - Average of vol_ratio_60
    stock_df['avg_log_vol_ratio_60'] = stock_df['vol_ratio_60'].rolling(window=60, min_periods=1).mean()
    
    # 7. std_log_vol_ratio_60 - Standard deviation of vol_ratio_60
    stock_df['std_log_vol_ratio_60'] = stock_df['vol_ratio_60'].rolling(window=60, min_periods=1).std()
    
    # Fill NaN 0으로
    stock_df[['vol_20', 'vol_60', 'vol_60_sqrt252', 'log_vol', 'vol_ratio_60', 
              'avg_log_vol_ratio_60', 'std_log_vol_ratio_60']] = stock_df[['vol_20', 'vol_60', 'vol_60_sqrt252', 
              'log_vol', 'vol_ratio_60', 'avg_log_vol_ratio_60', 'std_log_vol_ratio_60']].fillna(0)
    
    all_stocks_with_volume.append(stock_df)

# 모든 종목에 vol 추가
combined_df_final = pd.concat(all_stocks_with_volume, ignore_index=True)

print(f"\n✓ Added volume pattern features")
volume_cols = ['vol_20', 'vol_60', 'vol_60_sqrt252', 'log_vol', 'vol_ratio_60', 'avg_log_vol_ratio_60', 'std_log_vol_ratio_60']
print(f"Volume columns: {volume_cols}")
print(f"\nDataFrame shape: {combined_df_final.shape}")
print(f"Total columns: {len(combined_df_final.columns)}")

# 샘플 데이터
print(f"\n{'='*80}")
print(f"Sample: First 10 rows showing volume features")
print(f"{'='*80}")
sample = combined_df_final[combined_df_final['Ticker'] == combined_df_final['Ticker'].iloc[0]].head(10)
print(sample[['Date', 'Volume', 'log_vol', 'vol_20', 'vol_60', 'vol_ratio_60']])

Calculating volume pattern features...



Adding volume features: 100%|██████████| 200/200 [00:04<00:00, 45.35it/s]



✓ Added volume pattern features
Volume columns: ['vol_20', 'vol_60', 'vol_60_sqrt252', 'log_vol', 'vol_ratio_60', 'avg_log_vol_ratio_60', 'std_log_vol_ratio_60']

DataFrame shape: (471300, 24)
Total columns: 24

Sample: First 10 rows showing volume features
        Date   Volume    log_vol    vol_20    vol_60  vol_ratio_60
0 2015-01-02  2918197  14.886477  0.000000  0.000000      0.000000
1 2015-01-05  3563551  15.086268  0.000000  0.000000      0.099896
2 2015-01-06  2619991  14.778682  0.002395  0.002395     -0.138461
3 2015-01-07  1679442  14.333973  0.008145  0.008145     -0.437377
4 2015-01-08  2166767  14.588747  0.015169  0.015169     -0.146082
5 2015-01-09  1737936  14.368209  0.013844  0.013844     -0.305517
6 2015-01-12  1449448  14.186694  0.013666  0.013666     -0.417456
7 2015-01-13  1380551  14.137994  0.012845  0.012845     -0.407887
8 2015-01-14  2399519  14.690779  0.012654  0.012654      0.128799
9 2015-01-15  2884385  14.874823  0.012395  0.012395      0.281558


In [ ]:
# data 폴더 생성
os.makedirs('data', exist_ok=True)

print("Saving individual stock files with all features...\n")

for ticker in tqdm(combined_df_final['Ticker'].unique(), desc="Saving files"):
    # Ticker로 종목 선정
    stock_data = combined_df_final[combined_df_final['Ticker'] == ticker].copy()
    
    # Date index로 설정
    stock_data = stock_data.set_index('Date').sort_index()
    
    # Ticker 중복 column 제거
    stock_data = stock_data.drop('Ticker', axis=1)
    
    # ticker_df에서 종목명 추출
    stock_name = ticker_df[ticker_df.iloc[:, 0].astype(str).str.zfill(6) == ticker].iloc[:, 1].values[0] if len(ticker_df[ticker_df.iloc[:, 0].astype(str).str.zfill(6) == ticker]) > 0 else "Unknown"
    
    # 파일 이름 설정
    filename = f"{ticker}_{stock_name}.parquet"
    filepath = os.path.join('data', filename)
    
    # parquet으로 저장
    stock_data.to_parquet(filepath, compression='snappy')

print(f"\n✓ Saved {len(combined_df_final['Ticker'].unique())} individual stock files to 'data/' folder")
print(f"Each file has Date as index")
print(f"Columns per file: {len(stock_data.columns)} (OHLCV + Returns + Volume features)")

# 5종목 샘플
files = sorted(os.listdir('data'))
print(f"\nFirst 5 files:")
for f in files[:5]:
    print(f"  - {f}")

Saving individual stock files with all features...



Saving files: 100%|██████████| 200/200 [00:03<00:00, 54.36it/s]


✓ Saved 200 individual stock files to 'data/' folder
Each file has Date as index
Columns per file: 22 (OHLCV + Returns + Volume features)

First 5 files:
  - 000030_우리은행.parquet
  - 000070_삼양홀딩스.parquet
  - 000080_하이트진로.parquet
  - 000100_유한양행.parquet
  - 000120_CJ대한통운.parquet


In [ ]:
# 데이터 불러와지는 확인
import os
import pandas as pd

data_folder = 'data'
parquet_files = [f for f in os.listdir(data_folder) if f.endswith('.parquet')]

print(f"Found {len(parquet_files)} parquet files\n")

sample_file = parquet_files[0]
sample_df = pd.read_parquet(os.path.join(data_folder, sample_file))

print(f"Sample file: {sample_file}")
print(f"Shape: {sample_df.shape}")
print(f"\nColumns: {list(sample_df.columns)}")
print(f"\nFirst 10 rows:")
print(sample_df.head(10))

print(f"\n✓ Files saved with {len(sample_df.columns)} columns including returns!")

Found 200 parquet files

Sample file: 002240_고려제강.parquet
Shape: (2458, 22)

Columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Change', 'Return_1d', 'Return_5d', 'Return_20d', 'Return_30d', 'Return_50d', 'Return_60d', 'Return_100d', 'Return_120d', 'Return_200d', 'vol_20', 'vol_60', 'vol_60_sqrt252', 'log_vol', 'vol_ratio_60', 'avg_log_vol_ratio_60', 'std_log_vol_ratio_60']

First 10 rows:
             Open   High    Low  Close  Volume    Change  Return_1d  \
Date                                                                  
2015-01-02  27916  28017  27415  27617   27714 -0.021333   0.000000   
2015-01-05  27615  27715  26378  26916   34948 -0.025383  -2.538292   
2015-01-06  26715  27050  26481  26850   18115 -0.002452  -0.245207   
2015-01-07  26848  27216  26581  26783   20686 -0.002495  -0.249534   
2015-01-08  26981  27049  26681  26716   11595 -0.002502  -0.250159   
2015-01-09  26746  27514  26746  27383   11730  0.024966   2.496631   
2015-01-12  27115  27682  27049  2721